# Olist + DuckDB: Quick Exploration

This notebook connects to the local `olist.duckdb` database that you created with `duckdb_ingest.py` and runs a few sanity checks and exploratory queries.

**What you’ll do here:**  
1. Connect to DuckDB  
2. List tables & row counts  
3. Peek into orders, items, customers  
4. Produce a simple monthly sales plot

> Tip: If you need to re-run ingestion, execute `python duckdb_ingest.py` in your project root first.


In [ ]:
import duckdb, pandas as pd
import matplotlib.pyplot as plt

# Connect to local DuckDB database file (make sure olist.duckdb is in the same working directory)
con = duckdb.connect(database="olist.duckdb", read_only=True)

# Helper to show a query as a DataFrame
def q(sql):
    return con.execute(sql).df()

## Show tables

In [ ]:
q("SHOW TABLES")

## Basic row counts

In [ ]:
tables = [t for t in q("SHOW TABLES")['name'].tolist()]
counts = []
for t in tables:
    c = q(f"SELECT '{t}' AS table_name, COUNT(*) AS row_count FROM {t}")
    counts.append(c)
row_counts = pd.concat(counts, ignore_index=True)
row_counts

## Peek into key tables

In [ ]:
{
    "orders": q("SELECT * FROM orders LIMIT 5"),
    "order_items": q("SELECT * FROM order_items LIMIT 5"),
    "customers": q("SELECT * FROM customers LIMIT 5"),
    "products": q("SELECT * FROM products LIMIT 5")
}

## Monthly Sales Trend (price + freight)

In [ ]:
monthly = q(
    '''
    WITH items AS (
        SELECT
            o.order_purchase_timestamp::DATE AS purchase_date,
            DATE_TRUNC('month', o.order_purchase_timestamp)::DATE AS month_start,
            (oi.price + oi.freight_value) AS line_total
        FROM orders o
        JOIN order_items oi ON o.order_id = oi.order_id
        WHERE o.order_purchase_timestamp IS NOT NULL
    )
    SELECT month_start, SUM(line_total) AS monthly_revenue
    FROM items
    GROUP BY 1
    ORDER BY 1
    '''
)
monthly.set_index('month_start')['monthly_revenue'].plot(figsize=(10,4))
plt.title("Monthly Revenue (Olist)")
plt.xlabel("Month")
plt.ylabel("Revenue")
plt.show()

monthly.head()

## Top Product Categories by Revenue

In [ ]:
top_cat = q(
    '''
    SELECT
        COALESCE(ct.product_category_name_english, p.product_category_name) AS category_name,
        SUM(oi.price + oi.freight_value) AS revenue
    FROM order_items oi
    JOIN products p ON oi.product_id = p.product_id
    LEFT JOIN category_translation ct
      ON p.product_category_name = ct.product_category_name
    GROUP BY 1
    ORDER BY revenue DESC
    LIMIT 15
    '''
)
top_cat